Transformer(Bert)

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.3 MB/s eta 0:00:00


In [20]:
import json
import pandas as pd
import random
import pyarrow as pa
import datasets
import tqdm
import string
tqdm.tqdm.pandas()

### preprocessing and EDA

In [47]:
import pandas as pd
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go

In [49]:
maindf=pd.read_csv('emotion_data.csv')

Data is Imbalance so undersample to lower count data

In [17]:
from imblearn.under_sampling import RandomUnderSampler

In [18]:
rus = RandomUnderSampler(sampling_strategy='not minority', random_state=1)
df_balanced, balanced_labels = rus.fit_resample(maindf, maindf['emotion'])

In [48]:
fig = px.bar(df_balanced['emotion'].value_counts().rename_axis('هیجان').reset_index(name='تعداد'), x="هیجان", y="تعداد", color="تعداد")
fig.update_layout(title_text="توزیع داده هیجان", title_x=0.5)
pyo.iplot(fig, filename = 'توزیع داده هیجان')

In [21]:
def clean_data(text):
  english_alphabet=list(string.ascii_letters)
  numbers=list('0123456789۰١۲۳۴۵۶۷۸۹')
  for char in english_alphabet+numbers:
    text=text.replace(char,"")
  text=text.replace("/","")
  return text

In [22]:
df_balanced['tweet']=df_balanced['tweet'].progress_apply(lambda x:clean_data(x))

100%|██████████| 6948/6948 [00:00<00:00, 19724.61it/s]


In [23]:
from sklearn.model_selection import train_test_split

In [24]:
label2id={'joy':0,'sad':1,'anger':2,'disgust':3,'fear':4,'surprise':5}
df_balanced['emotion']=df_balanced['emotion'].apply(lambda x : label2id[x])

In [25]:
train_d,test_d =train_test_split(df_balanced,shuffle=True,test_size=0.2,stratify=df_balanced['emotion'])

In [26]:
test=[]
train=[]
for i in range(len(test_d)):
   td={'text':list(test_d['tweet'])[i],'label':list(test_d['emotion'])[i]}
   test.append(td)
for i in range(len(train_d)):
   td={'text':list(train_d['tweet'])[i],'label':list(train_d['emotion'])[i]}
   train.append(td)

In [27]:
train=pa.Table.from_arrays([[a['text'] for a in train],[a['label'] for  a in train]],names=['text','label'])
test=pa.Table.from_arrays([[a['text'] for a in test],[a['label'] for  a in test]],names=['text','label'])
datasetsDict={}
datasetsDict['train']=datasets.Dataset(train)
datasetsDict['test']=datasets.Dataset(test)
dts=datasets.DatasetDict(datasetsDict)

In [82]:
dts

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5558
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1390
    })
})

In [28]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.0 MB/s eta 0:00:00


In [29]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.4 MB/s eta 0:00:00


In [30]:
import torch
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification,TrainingArguments,Trainer

In [31]:
def gpu_check():
  torch.__version__
  #If ther 's a GPU available...
  if torch.cuda.is_available():
    #Tell Pytorch to use GPU.
    device = torch.device("cuda")
    print("There are %d GPU(s) avalilable." % torch.cuda.device_count())
    print('We will use the GPU:',torch.cuda.get_device_name(0))
    #If not..
  else:
    print('No GPU available,using the CPU instead.')
    device = torch.device("cpu")
    return device

In [32]:
device=gpu_check()
print(device)

There are 1 GPU(s) avalilable.
We will use the GPU: Tesla T4
None


In [33]:
label2id={'joy':0,'sad':1,'anger':2,'disgust':3,'fear':4,'surprise':5}
id2label={0:'joy',1:'sad',2:'anger',3:'disgust',4:'fear',5:'surprise'}

In [34]:
modelname="HooshvareLab/bert-base-parsbert-uncased"
tokenizer=AutoTokenizer.from_pretrained(modelname,model_max_length=100, add_special_tokens = True)
model=AutoModelForSequenceClassification.from_pretrained(
    modelname,num_labels=6,id2label=id2label,label2id=label2id
).to(device)
data_collector = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
from sklearn.metrics import precision_recall_fscore_support,accuracy_score

In [36]:
def preprocess_function(examples):
  return tokenizer(examples["text"],truncation=True)
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, fscore, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    accuracy=accuracy_score(labels, preds)

    # Check if support is None and replace it with 0
    # if support is None:
    #     support = 0

    metrics = {
        'precision': precision,
        'recall': recall,
        'fscore': fscore,
        # 'support': support,
        'accuracy': accuracy,
    }
    return metrics

In [37]:
tokenized_data=dts.map(preprocess_function,batched=True)

Map:   0%|          | 0/5558 [00:00<?, ? examples/s]

Map:   0%|          | 0/1390 [00:00<?, ? examples/s]

In [38]:
training_args = TrainingArguments(
    output_dir="output/",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy ="epoch",
    save_strategy ="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
)

In [39]:
trainer =Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collector,
    compute_metrics=compute_metrics
)

In [40]:
print("شروع آموزش...")
trainer.train()

شروع آموزش...


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,Fscore,Accuracy
1,No log,0.228144,0.934007,0.933813,0.933643,0.933813
2,0.529000,0.279313,0.936078,0.935252,0.934833,0.935252
3,0.134500,0.253381,0.946340,0.946043,0.945948,0.946043
4,0.134500,0.324297,0.943326,0.942446,0.942262,0.942446
5,0.055600,0.309366,0.947058,0.946763,0.946772,0.946763
6,0.015900,0.336745,0.946487,0.946043,0.945927,0.946043
7,0.015900,0.341998,0.948567,0.948201,0.948188,0.948201
8,0.007700,0.350839,0.946292,0.946043,0.945917,0.946043
9,0.004600,0.358917,0.947747,0.947482,0.947276,0.947482
10,0.004600,0.359672,0.947736,0.947482,0.947307,0.947482


TrainOutput(global_step=3480, training_loss=0.10775634704650133, metrics={'train_runtime': 1045.7821, 'train_samples_per_second': 53.147, 'train_steps_per_second': 3.328, 'total_flos': 2086983452282832.0, 'train_loss': 0.10775634704650133, 'epoch': 10.0})

In [41]:
trainer.evaluate()

{'eval_loss': 0.2281436175107956,
 'eval_precision': 0.9340065373914236,
 'eval_recall': 0.9338129496402877,
 'eval_fscore': 0.9336430880123312,
 'eval_accuracy': 0.9338129496402877,
 'eval_runtime': 6.589,
 'eval_samples_per_second': 210.957,
 'eval_steps_per_second': 13.204,
 'epoch': 10.0}

In [43]:
trainer.save_model('emotion')

In [44]:
!zip -r emotion_bert_model.zip 'emotion'

  adding: emotion/ (stored 0%)
  adding: emotion/special_tokens_map.json (deflated 42%)
  adding: emotion/vocab.txt (deflated 62%)
  adding: emotion/tokenizer.json (deflated 72%)
  adding: emotion/config.json (deflated 52%)
  adding: emotion/pytorch_model.bin (deflated 8%)
  adding: emotion/training_args.bin (deflated 49%)
  adding: emotion/tokenizer_config.json (deflated 47%)
